In [1]:
import pandas as pd
import os
os.listdir()
os.listdir("Data/")#Contents of the Data folder

['cz_item_category',
 'item_cz',
 'list_category',
 'material_attributes_cz',
 'product_views_cz']

In [2]:
dataFile = "Data/product_views_cz"
header_list = ['date','user_id_anon','product_id']
empData = pd.read_csv(dataFile,usecols=header_list, nrows = 4000000)
empData.head(10)
data_2020 = empData[empData['date'].str.startswith("2020")] #filtruje vzorek dat z roku 2020
print(data_2020)

               date  user_id_anon    product_id
0        2020-06-15       8805839       1047757
1        2020-06-16       8805839       1047757
2        2020-08-03       4278023  100005636515
3        2020-08-04       4278023  100005636515
4        2020-04-19      24560475       1097707
...             ...           ...           ...
3999873  2020-01-12      34736670       1216527
3999874  2020-03-13      55822494  100010839495
3999898  2020-10-05      12738986  100030604444
3999900  2020-04-08      30090264     493158001
3999908  2020-09-09      38796344  100020798417

[1067893 rows x 3 columns]


In [3]:
ids = data_2020['user_id_anon'] #filtruje users kteri se divali jen na jednu vec:
data_2020[ids.isin(ids[ids.duplicated()])].sort_values(['user_id_anon','date']) 

,date,user_id_anon,product_id
1858714,2020-01-05,249,1031619
1858715,2020-01-20,249,498788
1858716,2020-02-01,249,1470481
1858717,2020-03-07,249,1100588
1858718,2020-03-11,249,1664332
...,...,...,...
1024703,2020-07-04,62570517,906570
1024704,2020-07-19,62570517,1417467003
1024705,2020-07-19,62570517,1417467003
1024706,2020-07-19,62570517,1417607003


In [4]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [5]:
data_2020.dtypes

date            object
user_id_anon     int64
product_id       int64
dtype: object

In [7]:
data_2020 = data_2020.dropna().drop_duplicates()
print(len(data_2020))
data_2020 = data_2020[['user_id_anon','product_id']]

994754


In [8]:
# data_2020.head()
# print(data_2020[data_2020['user_id_anon'] == 889])

In [9]:
#Preprocessing dat (ve formatu list stringů)
data_2020['product_id'] = data_2020['product_id'].map(str) 
sentences = data_2020.groupby('user_id_anon')['product_id'].apply(list)
print(sentences)

user_id_anon
249              [1031619, 498788, 1470481, 1100588, 1664332]
374         [1261351, 1019492001, 662719, 100003953123, 40...
517                                            [100011270080]
889                                            [100013695593]
891                      [1115679, 1466497, 1046218, 1106568]
                                  ...                        
62569217                                            [2606430]
62569610                                            [1410630]
62569763                                             [582303]
62570325    [100031793420, 1417410, 493129001, 10001130498...
62570517    [1046692, 1427815, 1427815, 682803, 906570, 14...
Name: product_id, Length: 196673, dtype: object


In [10]:
import multiprocessing

from gensim.models import Word2Vec

cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_model = Word2Vec(min_count=2,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [11]:
t = time()

w2v_model.build_vocab(sentences, progress_per=5000) #vlozi slova do vocabulary

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:19:33: collecting all words and their counts
INFO - 21:19:33: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:19:33: PROGRESS: at sentence #5000, processed 23424 words, keeping 17893 word types
INFO - 21:19:33: PROGRESS: at sentence #10000, processed 49135 words, keeping 33784 word types
INFO - 21:19:34: PROGRESS: at sentence #15000, processed 74627 words, keeping 47125 word types
INFO - 21:19:34: PROGRESS: at sentence #20000, processed 98911 words, keeping 58870 word types
INFO - 21:19:34: PROGRESS: at sentence #25000, processed 124774 words, keeping 70679 word types
INFO - 21:19:34: PROGRESS: at sentence #30000, processed 151559 words, keeping 81756 word types
INFO - 21:19:34: PROGRESS: at sentence #35000, processed 176949 words, keeping 91569 word types
INFO - 21:19:34: PROGRESS: at sentence #40000, processed 202143 words, keeping 100519 word types
INFO - 21:19:34: PROGRESS: at sentence #45000, processed 226848 words, keeping 109103 word types
I

Time to build vocab: 0.75 mins


In [12]:
t = time()

#trenovani modelu:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:20:22: training model with 3 workers on 126394 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 21:20:23: EPOCH 1 - PROGRESS: at 11.26% examples, 90657 words/s, in_qsize 5, out_qsize 0
INFO - 21:20:25: EPOCH 1 - PROGRESS: at 24.90% examples, 100544 words/s, in_qsize 6, out_qsize 0
INFO - 21:20:26: EPOCH 1 - PROGRESS: at 36.49% examples, 98273 words/s, in_qsize 5, out_qsize 0
INFO - 21:20:27: EPOCH 1 - PROGRESS: at 50.64% examples, 101584 words/s, in_qsize 5, out_qsize 0
INFO - 21:20:28: EPOCH 1 - PROGRESS: at 63.83% examples, 101787 words/s, in_qsize 6, out_qsize 0
INFO - 21:20:29: EPOCH 1 - PROGRESS: at 77.94% examples, 103715 words/s, in_qsize 5, out_qsize 0
INFO - 21:20:30: EPOCH 1 - PROGRESS: at 90.97% examples, 103162 words/s, in_qsize 5, out_qsize 0
INFO - 21:20:30: worker thread finished; awaiting finish of 2 more threads
INFO - 21:20:30: worker thread finished; awaiting finish of 1 more threads
INFO - 21:20:31: worker thread finish

Time to train the model: 3.83 mins


In [13]:
w2v_model.init_sims(replace=True)

INFO - 21:24:39: precomputing L2-norms of word weight vectors


In [14]:
# K product_id daného v závorce jako string najde nejvhodnější jiné produkty:
w2v_model.wv.most_similar(positive=["1422386"])

[('1422379', 0.936413586139679),
 ('1336422', 0.8933944702148438),
 ('1336444', 0.8445572257041931),
 ('1378040', 0.8347022533416748),
 ('1439858', 0.8119312524795532),
 ('100018332643', 0.807379961013794),
 ('100018564173', 0.7933148145675659),
 ('100002200969', 0.7877377271652222),
 ('100021173465', 0.7817639708518982),
 ('100015798663', 0.7808905243873596)]

In [15]:
#Vypsání produktů, které jsou nejvícekrát obsazené ve vzorku tabulky product_views_cz:
temp = data_2020.groupby('product_id').apply(len)
print(temp[temp >= 200].sort_values())

product_id
845060           200
1709403          201
1339683          205
1375881          205
1024014          206
                ... 
100017733501     533
100022450196     568
1083932          661
1394534          723
843689          1423
Length: 78, dtype: int64


In [16]:
#Ulozi model do souboru
w2v_model.save("w2v.model")

INFO - 21:25:15: saving Word2Vec object under w2v.model, separately None
INFO - 21:25:15: storing np array 'vectors' to w2v.model.wv.vectors.npy
INFO - 21:25:17: not storing attribute vectors_norm
INFO - 21:25:17: storing np array 'syn1neg' to w2v.model.trainables.syn1neg.npy
INFO - 21:25:19: not storing attribute cum_table
INFO - 21:25:19: saved w2v.model
